In [ ]:
from __init__ import *

In [ ]:
signals = []

for data_dict_i, data_dict in enumerate(SETTING["model_data_dicts"]):

    signal = pd.read_csv(
        "../output/signal/{}.tsv".format(data_dict_i), sep="\t", index_col=0,
    )

    signal.columns.name = data_dict["axis_1_name"]

    signals.append(signal)

In [ ]:
model_mode = SETTING["model_mode"]

if model_mode == "range":

    axis_same = 0

    axis_different = 1

elif model_mode == "deep":

    axis_same = 1

    axis_different = 0

signals = kraft.dataframe.sync_axis(
    kraft.dataframe.sync_axis(signals, axis_same, "union"),
    axis_different,
    "symmetric_difference",
)

In [ ]:
model_mode = SETTING["model_mode"]

if model_mode == "range":

    method = "vs_w_hs"

elif model_mode == "deep":

    method = "vs_ws_h"

for r in SETTING["mf_rs"]:

    mf_r_directory_path = "../output/mf_r{}/".format(r)

    kraft.path.path(mf_r_directory_path)

    kraft.mf_dataframes(signals, method, r, mf_r_directory_path, plot_w_h=False)